In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog

In [2]:
# Function to collect room capacities
def collect_room_capacities():
    # Simulated room capacities
    room_capacities = [30] * 25 + [35] * 4 + [25]
    return room_capacities

In [3]:
# Room Assignment Heuristic
def assign_clusters_to_rooms(df, capacities):
    cluster_counts = df['Cluster'].value_counts().sort_index()
    room_assignments = {}
    room_id = 0
    for cluster, count in cluster_counts.items():
        while count > 0:
            available_capacity = capacities[room_id]
            room_assignments.setdefault(room_id, []).append(cluster)
            count -= available_capacity
            room_id = (room_id + 1) % len(capacities)
    return room_assignments

In [4]:
# Faculty Assignment to Rooms
def match_faculty_to_rooms(assignments, faculties):
    faculty_assignments = {}
    for room, clusters in assignments.items():
        common_domain = students_df[students_df['Cluster'].isin(clusters)]['Domain'].mode()
        if not common_domain.empty:
            faculty = faculties[faculties['Expertise_Domain'] == common_domain.iloc[0]]
            if not faculty.empty:
                faculty_assignments[room] = faculty.iloc[0]['Faculty_ID']
            else:
                faculty_assignments[room] = "No faculty available for Domain"
        else:
            faculty_assignments[room] = "No common domain found"
    return faculty_assignments

In [5]:
# Reporting Function
def generate_report(room_assignments, faculty_room_assignments):
    report = "Seating Plan and Faculty Assignments Report\n\n"
    report += "Room Assignments:\n"
    for room, clusters in room_assignments.items():
        report += f"Room {room}: Clusters {clusters}\n"
    report += "\nFaculty Assignments:\n"
    for room, faculty in faculty_room_assignments.items():
        report += f"Room {room}: Faculty ID {faculty}\n"
    return report

In [6]:
# Save Report to File
def save_report(report):
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", ".txt"), ("All files", ".*")])
    if file_path:
        with open(file_path, "w") as file:
            file.write(report)

In [7]:
# Tkinter Application Setup
root = tk.Tk()
root.title("Exam Management System")

# Simulating Student Data
data = {
    'Student_ID': range(1, 2501),
    'Domain': np.random.choice([0, 1, 2, 3, 4], 2500),  # CS=0, AI=1, BA=2, SE=3, EE=4
    'Batch': np.random.choice([19, 20, 21, 22, 23], 2500)
}
students_df = pd.DataFrame(data)

# K-Means Clustering
kmeans = KMeans(n_clusters=30, random_state=0)
students_df['Cluster'] = kmeans.fit_predict(students_df[['Domain', 'Batch']])

# Simulating Faculty Data
faculty_data = {
    'Faculty_ID': range(1,31),
    'Expertise_Domain': np.random.choice([0, 1, 2, 3, 4], 30)
}
faculty_df = pd.DataFrame(faculty_data)

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: ConvergenceWarning: Number of distinct clusters (25) found smaller than n_clusters (30). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [8]:
# Room capacities
room_capacities = collect_room_capacities()

# Assign clusters to rooms
room_assignments = assign_clusters_to_rooms(students_df, room_capacities)

# Faculty assignment to rooms
faculty_room_assignments = match_faculty_to_rooms(room_assignments, faculty_df)

# Generate report
report = generate_report(room_assignments, faculty_room_assignments)

# Seating Plan and Faculty Assignment Display
report_text = scrolledtext.ScrolledText(root, width=80, height=20)
report_text.pack()
report_text.insert(tk.INSERT, report)

# Button to Save Report
save_button = ttk.Button(root, text="Save Report", command=lambda: save_report(report))
save_button.pack()

root.mainloop()
